In [1]:
import pandas as pd
import numpy as np

# Part 1. Building the dataframe 

### Using read_html from pandas to read in tables from given html

In [2]:
toronto_raw = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [3]:
type(toronto_raw)

list

In [4]:
toronto_raw[0:1]

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                          Neighbourhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

### Looks like our table of interest is the first element in the list "toronto_raw" where we scraped the data to. I'll create a data frame from that element.

In [5]:
toronto = pd.DataFrame(toronto_raw[0])

In [6]:
toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
toronto.shape

(180, 3)

In [8]:
toronto["Borough"].unique()

array(['Not assigned', 'North York', 'Downtown Toronto', 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

### Looks like "Not assigned" is the only format of missing assignment in "Borough" column. Let's exclude all the rows where "Borough" is equal to "Not assigned"

In [9]:
toronto = toronto[toronto["Borough"] != "Not assigned"]

In [10]:
toronto[0:10]

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
len(toronto["Neighbourhood"][toronto["Neighbourhood"] == "Not assigned"])

0

### One of the rows has "Neighbourhood" = "Not assigned". Let's assign it to the name of it's "Borough"

In [12]:
toronto["Neighbourhood"][toronto["Neighbourhood"] == "Not assigned"] = toronto["Borough"]

In [13]:
toronto[0:10]

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [14]:
len(toronto["Neighbourhood"][toronto["Neighbourhood"] == "Not assigned"])

0

### In order to combine the value of "Neighbourhood" that have the same Postcode, I will use .groupby() and join value, separating them by a comma, and overwrite the "toronto" dataframe with the new dataframe where "Neighbourhood"s are listed in one row. Then I'll reset the index

In [15]:
toronto = pd.DataFrame(
    toronto.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(lambda hoods: ", ".join(hoods)))

KeyError: 'Postcode'

In [ ]:
toronto.reset_index(inplace=True)

In [ ]:
toronto.head()

### The shape of the final "toronto" dataframe is:

In [ ]:
toronto.shape

# Part 2. 

### Using the geocoder didn't work, the cell below would run for a very long time without any output, so I'm commenting it out and using the provided csv-file

In [ ]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [ ]:
import urllib.request
urllib.request.urlretrieve("http://cocl.us/Geospatial_data", "location_data.csv")

In [ ]:
location_data = pd.read_csv("location_data.csv")

In [ ]:
location_data.head()

In [ ]:
toronto = toronto.merge(location_data, left_on="Postcode", right_on="Postal Code", how="left")

In [ ]:
toronto.head()

In [ ]:
toronto.shape

In [ ]:
toronto.drop(["Postal Code"], axis=1, inplace=True)

In [ ]:
toronto.head()

In [ ]:
toronto.rename(columns={"Postcode": "PostalCode"}, inplace=True)

In [ ]:
toronto.head()

In [ ]:
toronto.shape

# Part 3. Clustering the Neighbourhoods

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

### Using geopy library to get the latitude and longitude values of Toronto.

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

### Create a map of Toronto with neighborhoods superimposed on top.

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Next I'll use Foursquare API to explore the neighbourhoods and segment them.
### Define Foursquare Credentials and Version.

In [ ]:
CLIENT_ID = 'XBKAMGZV4HLRWQMNFW3YEDMRI53AJKF3NZKMONCMNO1FAQDY' # your Foursquare ID
CLIENT_SECRET = 'VDMT1QTHUQK3V3P3S42FV2NQFKS50ZCX455T0UQAS4LEH5R0' # your Foursquare Secret
LIMIT = 30
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Now let's use the function from the course lab to explore all the neighbourhoods in Toronto.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now we use the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [ ]:
toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

### Let's check the size of the resulting dataframe

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

### Let's check how many venues were returned for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

## Preparing DataFrame for Clustering of Neighbourhoods

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

#move neighborhood column to the first column
fixed_columns = ["Neighborhood"] + list(toronto_onehot.columns.difference(['Neighborhood']))
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

In [ ]:
toronto_grouped.shape

### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Let's put that into a *pandas* dataframe. 
#### First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Finally Clustering!! 

### Run *k*-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=6).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype("int"))

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood').dropna()

toronto_merged.head() # check the last columns!

### Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Looks like there are two large clusters, but the other 3 are very small. Living in Toronto and knowing it's population and neighbourhood, I would say it probably makes some sense. Although Toronto is very heterogenous culturally, it's pretty homogenous commercially. There is acess to all sorts of venues in most of Neighbourhoods. 

### Let's have a look at two largest clusters, labels 0 and 1, and see if we can spot the difference.

### Here cluster labelled "0":

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

### And herem is cluster labelled "1":

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

### Let's see what are the most frequent venues in the "1st Most Common Venue" column for cluster "0":

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]["1st Most Common Venue"].value_counts()[0:10]

### Let's see what are the most frequent venues in the "1st Most Common Venue" column for cluster "1":

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]["1st Most Common Venue"].value_counts()

### Looks like cluster "0" has a lots of Coffee Shops and Restaurants, while cluster "1" has a lot of Parks, Convinience Stores and other suburban venues. That makes sense. 